# Spark Learning Note - Unsupervised Learning
Jia Geng | gjia0214@gmail.com


<a id='directory'></a>

## Directory

- [Data Source](https://github.com/databricks/Spark-The-Definitive-Guide/tree/master/data/)
- [1. Spark Unsupervised Learning Algorithms](#sec1)
- [2. k-Means](#sec2)
- [3. Bisect k-Means](#sec3)
- [4. Gaussian Mixture Model](#sec4)
- [5. Latent Dirichlet Allocation](#sec5)

## 1. Spark Unsupervised Learning Algorithms <a id='sec1'></a>

- k-Means
- Bisecting k-Means
- Gaussian Mixture Models
- Latent Dirichlet Allocation

[back to top](#directory)

In [1]:
from pyspark.sql.session import SparkSession

spark = SparkSession.builder.appName('MLexample').getOrCreate()
spark

In [3]:
data_path = '/home/jgeng/Documents/Git/SparkLearning/book_data/retail-data/by-day' 
sales = spark.read.format('csv').option('header', True).option('inferSchema', True).load(data_path)  
sales.cache()


541909

In [9]:
# explore the data
total = sales.count()
sales.printSchema()
sales.show(1)
for col_name in sales.columns:
    null_count = sales.where('{} is null'.format(col_name)).count()
    print("{} column num of nulls: {}/{}".format(col_name, null_count, total))

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|       Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
+---------+---------+------------------+--------+-------------------+---------+----------+--------------+
only showing top 1 row

InvoiceNo column num of nulls: 0/541909
StockCode column num of nulls: 0/541909
Description column num of 

DataFrame[summary: string, InvoiceNo: string, StockCode: string, Description: string, Quantity: string, UnitPrice: string, CustomerID: string, Country: string]

In [10]:
sales.summary().show()

+-------+------------------+------------------+--------------------+-----------------+-----------------+------------------+-----------+
|summary|         InvoiceNo|         StockCode|         Description|         Quantity|        UnitPrice|        CustomerID|    Country|
+-------+------------------+------------------+--------------------+-----------------+-----------------+------------------+-----------+
|  count|            541909|            541909|              540455|           541909|           541909|            406829|     541909|
|   mean|  559965.752026781|27623.240210938104|             20713.0| 9.55224954743324|4.611113626089641|15287.690570239585|       null|
| stddev|13428.417280796697|16799.737628427683|                 NaN|218.0811578502335|96.75985306117963| 1713.600303321597|       null|
|    min|            536365|             10002| 4 PURPLE FLOCK D...|           -80995|        -11062.06|           12346.0|  Australia|
|    25%|          547906.0|           21929.0| 

In [22]:
from pyspark.ml.feature import VectorAssembler

# assemble data into a vector
sales_vec = VectorAssembler().setInputCols(['Quantity', 'UnitPrice']).setOutputCol('features')\
                            .transform(sales.na.drop()).select('InvoiceNo', 'CustomerID', 'features')
sales_vec.show(1)

+---------+----------+-----------+
|InvoiceNo|CustomerID|   features|
+---------+----------+-----------+
|   580538|   14075.0|[48.0,1.79]|
+---------+----------+-----------+
only showing top 1 row



## 2. k-Means <a id='sec2'></a>

**k-means clustering is an NP-hard optimization problem.** A good approximation is the k-means++ algorithm. https://en.wikipedia.org/wiki/K-means%2B%2B.

> The exact algorithm is as follows:
- Choose one center uniformly at random among the data points.
- For each data point x, compute D(x), the distance between x and the nearest center that has already been chosen.
- Choose one new data point at random as a new center, using a weighted probability distribution where a point x is chosen with probability proportional to D(x)2.
- Repeat Steps 2 and 3 until k centers have been chosen.
- Now that the initial centers have been chosen, proceed using standard k-means clustering.


**Model Hyperparams**

The most important parameter is k.

|Param|Input|Notes|
|-|-|-|
|k|int|number of clusters, default is 2
|initMode|'random', 'k-means'|default is 'k-means', a parallel variant of k-means++
|initSteps|int|default is 2, number of steps for k-means
|maxIter|int|default is 2, max number of iterations
|tol|float|convergence tolerance for iterative algorithm, default is 1e-4


[back to top](#directory)

In [11]:
from pyspark.ml.clustering import KMeans

print(KMeans().explainParams())

distanceMeasure: the distance measure. Supported options: 'euclidean' and 'cosine'. (default: euclidean)
featuresCol: features column name. (default: features)
initMode: The initialization algorithm. This can be either "random" to choose random points as initial cluster centers, or "k-means||" to use a parallel variant of k-means++ (default: k-means||)
initSteps: The number of steps for k-means|| initialization mode. Must be > 0. (default: 2)
k: The number of clusters to create. Must be > 1. (default: 2)
maxIter: max number of iterations (>= 0). (default: 20)
predictionCol: prediction column name. (default: prediction)
seed: random seed. (default: -3718451565329112106)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 0.0001)


In [67]:
kmeans = KMeans().setK(100).setInitSteps(50).setMaxIter(500)
kmmodel = kmeans.fit(sales_vec)

In [68]:
summary = kmmodel.summary
??summary

In [69]:
kmmodel.transform(sales_vec).show(3)
print(summary.trainingCost)
kmmodel.computeCost(sales_vec)

+---------+----------+-----------+----------+
|InvoiceNo|CustomerID|   features|prediction|
+---------+----------+-----------+----------+
|   580538|   14075.0|[48.0,1.79]|        73|
|   580538|   14075.0|[20.0,1.25]|        94|
|   580538|   14075.0|[24.0,1.65]|        14|
+---------+----------+-----------+----------+
only showing top 3 rows

1683826.9533015746


1683826.9533015783

In [71]:
centers = kmmodel.clusterCenters()

## 3. Bisect k-Means <a id='sec3'></a>

Instead of the bottom-up approach, use top down approach: start with a single group then keep splitting the group until reach to the target number of groups. **Usually faster than the k-means but yields different results**.

APIs are pretty much the same as K-Means

**Model Hyperparams**

|Param|Input|Notes|
|-|-|-|
|k|int|default is 4
|maxIter|int|max number of iterations, default 20
|minDivisibleClusterSize|int|The minimum number of points of a divisible cluster

[back to top](#directory)

In [72]:
from pyspark.ml.clustering import BisectingKMeans
print(BisectingKMeans().explainParams())

distanceMeasure: the distance measure. Supported options: 'euclidean' and 'cosine'. (default: euclidean)
featuresCol: features column name. (default: features)
k: The desired number of leaf clusters. Must be > 1. (default: 4)
maxIter: max number of iterations (>= 0). (default: 20)
minDivisibleClusterSize: The minimum number of points (if >= 1.0) or the minimum proportion of points (if < 1.0) of a divisible cluster. (default: 1.0)
predictionCol: prediction column name. (default: prediction)
seed: random seed. (default: -1116325660993990397)


## 4. Gaussian Mixture Model <a id='sec4'></a>

Goal is to reduce the sum of the squared distance from the center of the cluster. **GMM assume the data should be result from random draw from multiple Gaussian components. I.e., for each Gaussian Component, the data within it should be normally distributed**. Each Gaussian component can have different means, stds so that it could have differen

t size and ellipsoid shape. 

**GMM is like a soft version of k-means. k-means creates a very rigid clusters, each point is only within one cluster. GMM allow for a more nuanced cluster associated with probabilities, instead of rigid boundaries.**

**Model Hyperparams**

|Param|Input|Notes|
|-|-|-|
|k|int|default is 4. most important
|maxIter|int|max number of iterations, default 100. does not affect performace much
|tol|float|convergence tolerance for iterative algorithm, default is 1e-2

[back to top](#directory)

In [76]:
from pyspark.ml.clustering import GaussianMixture

print(GaussianMixture().explainParams())

featuresCol: features column name. (default: features)
k: Number of independent Gaussians in the mixture model. Must be > 1. (default: 2)
maxIter: max number of iterations (>= 0). (default: 100)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
seed: random seed. (default: -1473277178213100281)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 0.01)


In [77]:
gmm = GaussianMixture().setK(20)
gmm_model = gmm.fit(sales_vec)

In [88]:
gmm_model.transform(sales_vec).show(3)  # proba is just a confidence of belonging to each Gaussian

+---------+----------+-----------+----------+--------------------+
|InvoiceNo|CustomerID|   features|prediction|         probability|
+---------+----------+-----------+----------+--------------------+
|   580538|   14075.0|[48.0,1.79]|        18|[1.99138136793025...|
|   580538|   14075.0|[20.0,1.25]|        18|[1.59393645061588...|
|   580538|   14075.0|[24.0,1.65]|        18|[1.61441707715752...|
+---------+----------+-----------+----------+--------------------+
only showing top 3 rows



In [94]:
# get the gaussian componenets
gmm_model.gaussiansDF.show(3)


+--------------------+--------------------+
|                mean|                 cov|
+--------------------+--------------------+
|[162.380218368691...|4.197210910517039...|
|[162.380218368691...|4.197210910517039...|
|[162.380218368942...|4.197210910517145...|
+--------------------+--------------------+
only showing top 3 rows



In [96]:
??gmm_model.weights
gmm_model.weights

[7.282617369738237e-07,
 7.282617369738237e-07,
 7.282617369739251e-07,
 7.282617369738237e-07,
 7.282617369738242e-07,
 7.282617369738255e-07,
 7.282617369738236e-07,
 7.282617369738237e-07,
 7.282617369738237e-07,
 7.282617369738237e-07,
 0.0001246244522596864,
 7.282617369738237e-07,
 7.282617370453267e-07,
 7.282617369738251e-07,
 7.282617369738242e-07,
 7.282617369738237e-07,
 7.282617369738236e-07,
 7.282617369738259e-07,
 0.9959187889813561,
 0.003944206116855548]

In [92]:
# get the clustering results
summary = gmm_model.summary
summary.cluster.show(3)  # clustering result
summary.probability.show(3) # proba results

+----------+
|prediction|
+----------+
|        18|
|        18|
|        18|
+----------+
only showing top 3 rows

+--------------------+
|         probability|
+--------------------+
|[1.99138136793025...|
|[1.59393645061588...|
|[1.61441707715752...|
+--------------------+
only showing top 3 rows



## 5. Latent Dirichlet Allocation <a id='sec5'></a>

Latent Dirichlet Allocation is a hierarchical clustering model usually used to perform topic modeling on text documents. LDA tries to extract high-level topics of documents and keywords associated with those topics. It then inteprets each documents as having a variable number of contirbutions from multiple input topics. There are two implementations: 
- online LDA
- EM (expectation maximization) LDA

https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation

**Model Hyperparams**

|Param|Input|Notes|
|-|-|-|
|docConcentration||Commonly named as alppha. Prior for dcouments's distributions over topics (as regularization). If not specified, will be set automatically
|topicConcentration||Commonly named as beta or eta. Prior for topic's distribution over terms. If not specified, will be set automatically
|maxIter|int|max number of iterations, default 20. does not affect performace much
|optimization|'em', 'online'|Default is 'online'. Onelin usually outperform em.
|learningDecay|float (0.5, 1.0]|Exponential decay rate. Default is 0.51.
|learningOffset|float|A (positive) learning parameter that downweights early iterations. Larger values make early iterations count less (default: 1024.0)
|subsamplingRate|float|Fraction of the corpus to be sampled and used in each iteration of mini-batch gradient descent, in range (0, 1]. (default: 0.05)

**Prediction Parameter**

|Param|Input|Notes|
|-|-|-|
|topicDistributionCol|str|The column that will hold the output of the topic mixture distribution for each document

[back to top](#directory)

In [97]:
from pyspark.ml.clustering import LDA

print(LDA().explainParams())

checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
docConcentration: Concentration parameter (commonly named "alpha") for the prior placed on documents' distributions over topics ("theta"). (undefined)
featuresCol: features column name. (default: features)
k: The number of topics (clusters) to infer. Must be > 1. (default: 10)
keepLastCheckpoint: (For EM optimizer) If using checkpointing, this indicates whether to keep the last checkpoint. If false, then the checkpoint will be deleted. Deleting the checkpoint can cause failures if a data partition is lost, so set this bit with care. (default: True)
learningDecay: Learning rate, set as anexponential decay rate. This should be between (0.5, 1.0] to guarantee asymptotic convergence. (default: 0.51)
learningOffset: A (positive) learni

In [136]:
from pyspark.ml.feature import Tokenizer, CountVectorizer

# transform description
# don't forget to remove the nulls
tkn = Tokenizer().setInputCol('Description').setOutputCol('tokens')
tknDF = tkn.transform(sales.na.drop()).select('InvoiceNo', 'tokens')
tknDF.show(1)

# convert to coutn vectors
# must renamed to features
counter = CountVectorizer().setInputCol('tokens').setOutputCol('features')\
                            .setVocabSize(500).setBinary(True)    
counterModel = counter.fit(tknDF)
cntDF = counterModel.transform(tknDF)
cntDF.show(1)

# clustering using lda
lda = LDA().setK(5).setMaxIter(10)
ldaModel = lda.fit(cntDF)


+---------+--------------------+
|InvoiceNo|              tokens|
+---------+--------------------+
|   580538|[rabbit, night, l...|
+---------+--------------------+
only showing top 1 row

+---------+--------------------+--------------------+
|InvoiceNo|              tokens|            features|
+---------+--------------------+--------------------+
|   580538|[rabbit, night, l...|(500,[158,199,224...|
+---------+--------------------+--------------------+
only showing top 1 row



In [137]:
# top 5 topics
ldaModel.describeTopics().show()

+-----+--------------------+--------------------+
|topic|         termIndices|         termWeights|
+-----+--------------------+--------------------+
|    0|[4, 20, 19, 23, 8...|[0.04680889359272...|
|    1|[0, 1, 2, 4, 9, 1...|[0.05224728817550...|
|    2|[1, 11, 5, 2, 22,...|[0.06127403098198...|
|    3|[36, 39, 44, 0, 1...|[0.03872941707759...|
|    4|[14, 21, 3, 32, 0...|[0.03875448353203...|
+-----+--------------------+--------------------+



In [138]:
pandaDF = ldaModel.describeTopics().toPandas()
print(pandaDF.head())

   topic                              termIndices  \
0      0     [4, 20, 19, 23, 8, 7, 2, 58, 62, 49]   
1      1       [0, 1, 2, 4, 9, 12, 15, 6, 27, 26]   
2      2       [1, 11, 5, 2, 22, 3, 28, 10, 6, 9]   
3      3  [36, 39, 44, 0, 17, 68, 53, 101, 5, 89]   
4      4    [14, 21, 3, 32, 0, 18, 63, 13, 5, 54]   

                                         termWeights  
0  [0.046808893592722944, 0.03858196436208082, 0....  
1  [0.05224728817550236, 0.04286418530247717, 0.0...  
2  [0.061274030981988366, 0.053000593467125816, 0...  
3  [0.038729417077595446, 0.03786740361476374, 0....  
4  [0.03875448353203486, 0.037391445284121416, 0....  


In [139]:
vocab = counterModel.vocabulary
print(vocab[:20])

['set', 'of', 'bag', 'red', 'heart', 'retrospot', 'vintage', 'design', 'pink', 'christmas', 'box', 'cake', '', 'white', 'metal', 'jumbo', 'lunch', '3', 'blue', 'hanging']


In [140]:
for i in range(5):
    topic = pandaDF.loc[i]['termIndices']
    content = [vocab[i] for i in topic]
    print('Topic{}\'s content is {}'.format(i, content))

Topic0's content is ['heart', 'holder', 'hanging', 't-light', 'pink', 'design', 'bag', 'feltcraft', 'assorted', 'doormat']
Topic1's content is ['set', 'of', 'bag', 'heart', 'christmas', '', 'jumbo', 'vintage', '6', 'wooden']
Topic2's content is ['of', 'cake', 'retrospot', 'bag', 'pack', 'red', 'cases', 'box', 'vintage', 'christmas']
Topic3's content is ['bottle', 'hot', 'water', 'set', '3', 'kit', 'cream', 'warmer', 'retrospot', 'hand']
Topic4's content is ['metal', 'sign', 'red', 'tea', 'set', 'blue', 'clock', 'white', 'retrospot', 'bunting']
